In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
import numpy as np
import gc

/Users/ruiqianyang/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_9.4.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
####data from r
train_r=pd.read_csv('train_r.csv')
test_r=pd.read_csv('test_r.csv')

#del X,test_x
X=train_r.drop(train_r.columns[0],axis=1)
test_x=test_r.drop(test_r.columns[0],axis=1)

label = pd.read_pickle('train_Basic.pkl')
y = label['isFraud']

train=pd.read_pickle('train_Basic.pkl')
test=pd.read_pickle('test_Basic.pkl')
y=y.reset_index(drop=False)
train=train.reset_index(drop=False)
test=test.reset_index(drop=False)

X['TransactionID']=train['TransactionID']
test_x['TransactionID']=test['TransactionID']

X['month']=train['month']
test_x['month']=test['month']

import datetime
startDT = datetime.datetime.strptime('2017-12-01', '%Y-%m-%d')
train['TransactionDT1']=train['TransactionDT'].apply(lambda x:(startDT+datetime.timedelta(seconds=x)))
X['DT_M'] = ((train['TransactionDT1'].dt.year-2017)*12 + train['TransactionDT1'].dt.month).astype(np.int8)
X['DT_M']


for df in [X, test_x]:
    for col_1 in df.columns:
        if df[col_1].dtype == 'object':
            print(col_1)
            le = LabelEncoder()
            le.fit(list(train_r[col_1].astype(str).values) + 
                   list(test_r[col_1].astype(str).values))
            df[col_1] = le.transform(list(df[col_1].astype(str).values))
    df.fillna(-999,inplace=True)
    df[np.isinf(df.values)==True] = 999

In [ ]:
###minify data
X_12=X[X['month']==12]
y_12=label[label['month']==12]['isFraud']

In [ ]:
##############################XGBOOST kfold##################################
import xgboost as xgb
from sklearn.model_selection import KFold,GroupKFold
import time
start=time.time()
       
def make_predictions(tr_df, tt_df, features_columns, target, 
                      NFOLDS=10):
    
    #folds = GroupKFold(n_splits=NFOLDS)  #GroupKFold by XX
    folds = KFold(n_splits=NFOLDS, shuffle=False, random_state=42)

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    #split_groups = tr_df['DT_M']      #GroupKFold by XX

    tt_df = tt_df[['TransactionID',target]] 
    #tt_df = tt_df[target]   
    predictions = np.zeros(len(tt_df))
    
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
   # for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)): #GroupKFold by XX
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
#         tr_data = xgb.DMatrix(tr_x, label=tr_y)
#         vl_data = xgb.DMatrix(vl_x, label=vl_y)
#         P_D = xgb.DMatrix(P)
        
 #       print('train month is '+str(tr_x['month'].unique()))
 #       print('test month is '+str(vl_x['month'].unique()))
#        tr_data = lgb.Dataset(tr_x, label=tr_y)

#         if LOCAL_TEST:
#             vl_data = lgb.Dataset(P, label=P_y) 
#         else:
#             vl_data = lgb.Dataset(vl_x, label=vl_y)  

#         estimator = xgb.XGBClassifier.train(
#             xgb_params,
#             tr_data,
#             evals = [(tr_data, 'train'),(vl_data, 'valid')], 
#             verbose_eval = 50
#         ) 

        clf = xgb.XGBClassifier(
             n_jobs=4,
            n_estimators=2000,
            max_depth=20,
            learning_rate=0.035,
            subsample=0.7,
            colsample_bytree=0.7463058454739352,
            #tree_method='gpu_hist',  # THE MAGICAL PARAMETER,
            random_state=2019,
            reg_alpha=0.39871702770778467,
            reg_lamdba=0.24309304355829786
#                gamma=0.6665437467229817,
#                min_child_weight=,
#                max_delta_step=,
#           min_child_samples= 170
        )
    
        clf.fit(tr_x, tr_y,eval_set = [(tr_x, tr_y),(vl_x, vl_y)], eval_metric="auc",
                early_stopping_rounds = 50)
        print(time.time()-start)
        print('ROC accuracy: {}'.format(roc_auc_score(vl_y, clf.predict_proba(vl_x)[:, 1])))
        #pp_p+= clf.predict_proba(P)[:,1] 
     
        pp_p = clf.predict_proba(P)[:,1]
        
        print(pp_p)
        tt_df['fold '+str(fold_)]=pp_p
        predictions += pp_p/NFOLDS     #consider weighting predictions??
       # print('weight is '+str(w[fold_]))
       # predictions += pp_p*w[fold_]
        
        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), 
                                       columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y
        gc.collect()
    
    tt_df['prediction'] = predictions
    
    return tt_df   

########################### Model Train
LOCAL_TEST = False
TARGET = 'isFraud'

#assign train_df and test_df:
train_df=X.iloc[:]
train_df['isFraud']=y['isFraud']
test_df=test_x.iloc[:]
test_df['isFraud']=0

#!!!if transactionID is index: 
train_df=train_df.reset_index(drop=False)
test_df=test_df.reset_index(drop=False)
    
#make sure 'DT_M' and'TransactionID' are not in feature list
features_columns=list(set(X)-{'DT_M','TransactionID'})     
print(len(features_columns))
print(len(X.columns))

test_predictions = make_predictions(train_df, test_df, features_columns, TARGET,NFOLDS=10)
####End XGB KFOLD

227
229
Fold: 0
472432 118108
[0]	validation_0-auc:0.879091	validation_1-auc:0.847155
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.894005	validation_1-auc:0.845267
[2]	validation_0-auc:0.902255	validation_1-auc:0.850384
[3]	validation_0-auc:0.905102	validation_1-auc:0.852482
[4]	validation_0-auc:0.908332	validation_1-auc:0.856521
[5]	validation_0-auc:0.910613	validation_1-auc:0.859506
[6]	validation_0-auc:0.915004	validation_1-auc:0.862178
[7]	validation_0-auc:0.915699	validation_1-auc:0.862331
[8]	validation_0-auc:0.916194	validation_1-auc:0.862227
[9]	validation_0-auc:0.917422	validation_1-auc:0.861611
[10]	validation_0-auc:0.919223	validation_1-auc:0.862658
[11]	validation_0-auc:0.921672	validation_1-auc:0.864194
[12]	validation_0-auc:0.92311	validation_1-auc:0.865486
[13]	validation_0-auc:0.924997	validation_1-auc:0.86625
[14]	validation_0-auc:0.927

[141]	validation_0-auc:0.999026	validation_1-auc:0.908743
[142]	validation_0-auc:0.999078	validation_1-auc:0.908745
[143]	validation_0-auc:0.999124	validation_1-auc:0.90884
[144]	validation_0-auc:0.999162	validation_1-auc:0.90901
[145]	validation_0-auc:0.999201	validation_1-auc:0.909031
[146]	validation_0-auc:0.999237	validation_1-auc:0.909244
[147]	validation_0-auc:0.999262	validation_1-auc:0.909173
[148]	validation_0-auc:0.999304	validation_1-auc:0.909206
[149]	validation_0-auc:0.99934	validation_1-auc:0.909433
[150]	validation_0-auc:0.999367	validation_1-auc:0.909478
[151]	validation_0-auc:0.999394	validation_1-auc:0.909631
[152]	validation_0-auc:0.999422	validation_1-auc:0.90972
[153]	validation_0-auc:0.999448	validation_1-auc:0.909906
[154]	validation_0-auc:0.999476	validation_1-auc:0.909936
[155]	validation_0-auc:0.999497	validation_1-auc:0.910077
[156]	validation_0-auc:0.99952	validation_1-auc:0.910137
[157]	validation_0-auc:0.99954	validation_1-auc:0.910159
[158]	validation_0-a

/Users/ruiqianyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold: 1
472432 118108
[0]	validation_0-auc:0.878523	validation_1-auc:0.842593
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.890128	validation_1-auc:0.855975
[2]	validation_0-auc:0.898155	validation_1-auc:0.859606
[3]	validation_0-auc:0.899785	validation_1-auc:0.861512
[4]	validation_0-auc:0.903683	validation_1-auc:0.863945
[5]	validation_0-auc:0.905713	validation_1-auc:0.86682
[6]	validation_0-auc:0.908343	validation_1-auc:0.868858
[7]	validation_0-auc:0.909361	validation_1-auc:0.869586
[8]	validation_0-auc:0.910728	validation_1-auc:0.87023
[9]	validation_0-auc:0.913588	validation_1-auc:0.871322
[10]	validation_0-auc:0.914529	validation_1-auc:0.871851
[11]	validation_0-auc:0.916091	validation_1-auc:0.872762
[12]	validation_0-auc:0.917564	validation_1-auc:0.873574
[13]	validation_0-auc:0.919815	validation_1-auc:0.875593
[14]	validation_0-auc:0.921758	vali

[141]	validation_0-auc:0.998558	validation_1-auc:0.929072
[142]	validation_0-auc:0.998616	validation_1-auc:0.929399
[143]	validation_0-auc:0.998684	validation_1-auc:0.929433
[144]	validation_0-auc:0.998735	validation_1-auc:0.929486
[145]	validation_0-auc:0.998786	validation_1-auc:0.929582
[146]	validation_0-auc:0.998845	validation_1-auc:0.929499
[147]	validation_0-auc:0.9989	validation_1-auc:0.929567
[148]	validation_0-auc:0.998951	validation_1-auc:0.929728
[149]	validation_0-auc:0.999008	validation_1-auc:0.930164
[150]	validation_0-auc:0.999059	validation_1-auc:0.930467
[151]	validation_0-auc:0.999097	validation_1-auc:0.930451
[152]	validation_0-auc:0.999141	validation_1-auc:0.930547
[153]	validation_0-auc:0.99918	validation_1-auc:0.930765
[154]	validation_0-auc:0.999224	validation_1-auc:0.930804
[155]	validation_0-auc:0.99926	validation_1-auc:0.930967
[156]	validation_0-auc:0.999288	validation_1-auc:0.931191
[157]	validation_0-auc:0.999314	validation_1-auc:0.931123
[158]	validation_0

[283]	validation_0-auc:0.999996	validation_1-auc:0.938633
[284]	validation_0-auc:0.999996	validation_1-auc:0.938637
[285]	validation_0-auc:0.999996	validation_1-auc:0.938676
[286]	validation_0-auc:0.999996	validation_1-auc:0.938656
Stopping. Best iteration:
[276]	validation_0-auc:0.999995	validation_1-auc:0.938812

15631.064803123474
ROC accuracy: 0.9388116845370211
[0.00058374 0.00069225 0.00118059 ... 0.00189566 0.00375809 0.00256558]


/Users/ruiqianyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold: 2
472432 118108
[0]	validation_0-auc:0.87777	validation_1-auc:0.836748
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.887526	validation_1-auc:0.857287
[2]	validation_0-auc:0.891486	validation_1-auc:0.861067
[3]	validation_0-auc:0.897004	validation_1-auc:0.866336
[4]	validation_0-auc:0.901919	validation_1-auc:0.86977
[5]	validation_0-auc:0.904285	validation_1-auc:0.871352
[6]	validation_0-auc:0.908292	validation_1-auc:0.873514
[7]	validation_0-auc:0.910068	validation_1-auc:0.874353
[8]	validation_0-auc:0.911099	validation_1-auc:0.875731
[9]	validation_0-auc:0.912812	validation_1-auc:0.877732
[10]	validation_0-auc:0.914782	validation_1-auc:0.878937
[11]	validation_0-auc:0.91611	validation_1-auc:0.880095
[12]	validation_0-auc:0.917369	validation_1-auc:0.880409
[13]	validation_0-auc:0.919932	validation_1-auc:0.88187
[14]	validation_0-auc:0.921875	valida

In [ ]:
#XGB GROUP kfold##################################
import xgboost as xgb
from sklearn.model_selection import KFold,GroupKFold
import time
start=time.time()
       
def make_predictions(tr_df, tt_df, features_columns, target, 
                      NFOLDS=7):
    
    folds = GroupKFold(n_splits=NFOLDS)  #GroupKFold by DT_M

    X,y = tr_df[features_columns], tr_df[target]    
    P,P_y = tt_df[features_columns], tt_df[target]  
    split_groups = tr_df['DT_M']      #GroupKFold by XX

    tt_df = tt_df[['TransactionID',target]] 
    predictions = np.zeros(len(tt_df))
    
 
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y, groups=split_groups)): #GroupKFold by XX
        print('Fold:',fold_)
        tr_x, tr_y = X.iloc[trn_idx,:], y[trn_idx]
        vl_x, vl_y = X.iloc[val_idx,:], y[val_idx]
            
        print(len(tr_x),len(vl_x))
#         tr_data = xgb.DMatrix(tr_x, label=tr_y)
#         vl_data = xgb.DMatrix(vl_x, label=vl_y)
#         P_D = xgb.DMatrix(P)
        
        print('train month is '+str(tr_x['month'].unique()))
        print('test month is '+str(vl_x['month'].unique()))
#        tr_data = lgb.Dataset(tr_x, label=tr_y)

#         if LOCAL_TEST:
#             vl_data = lgb.Dataset(P, label=P_y) 
#         else:
#             vl_data = lgb.Dataset(vl_x, label=vl_y)  

        clf = xgb.XGBClassifier(
             n_jobs=4,
            n_estimators=2000,
            max_depth=20,
            learning_rate=0.035,
            subsample=0.7,
            colsample_bytree=0.7463058454739352,
            #tree_method='gpu_hist',  # THE MAGICAL PARAMETER
            reg_alpha=0.39871702770778467,
            reg_lamdba=0.24309304355829786,
            random_state=2019
#                gamma=0.6665437467229817,
#                min_child_weight=,
#                max_delta_step=,
#           min_child_samples= 170
        )
    
        clf.fit(tr_x, tr_y,eval_set = [(tr_x, tr_y),(vl_x, vl_y)], eval_metric="auc",
                early_stopping_rounds = 50)
        print(time.time()-start)
        print('ROC accuracy: {}'.format(roc_auc_score(vl_y, clf.predict_proba(vl_x)[:, 1])))
        #pp_p+= clf.predict_proba(P)[:,1] 
     
        pp_p = clf.predict_proba(P)[:,1]
        
        print(pp_p)
        tt_df['fold '+str(fold_)]=pp_p
        predictions += pp_p/NFOLDS     #consider weighting predictions??
       # print('weight is '+str(w[fold_]))
       # predictions += pp_p*w[fold_]
        
        if LOCAL_TEST:
            feature_imp = pd.DataFrame(sorted(zip(estimator.feature_importance(),X.columns)), 
                                       columns=['Value','Feature'])
            print(feature_imp)
        
        del tr_x, tr_y, vl_x, vl_y
        gc.collect()
    
    tt_df['prediction'] = predictions
    
    return tt_df   

########################### Model Train
LOCAL_TEST = False
TARGET = 'isFraud'

#assign train_df and test_df:
train_df=X.iloc[:]
train_df['isFraud']=y['isFraud']
test_df=test_x.iloc[:]
test_df['isFraud']=0
train_df=train_df.reset_index(drop=False)
test_df=test_df.reset_index(drop=False)
    
#make sure 'DT_M' and'TransactionID' are not in feature list
features_columns=list(set(X)-{'DT_M','TransactionID'})     
print(len(features_columns))
print(len(X.columns))

test_predictions = make_predictions(train_df, test_df, features_columns, TARGET,NFOLDS=7)
####End GROUPKFOLD

In [3]:
test_predictions.head()

In [29]:
#important to reset index
test_x=test_x.reset_index(drop=False)

test_predictions.to_csv("XGB_Kfold_predictions_every_fold.csv",index=False)

#for kfold/groupkfold/stratifiedkfold
test_x['isFraud']=test_predictions['prediction']


In [31]:
output=test_x[['TransactionID','isFraud']]
#output['isFraud']=test_x['isFraud']

In [32]:
#test.shape
output.to_csv("output_submission_XGB_Kfold.csv",index=False)

#output1=output.copy()
#output1['isFraud']=test_predictions['fold 9']
#output1.to_csv("output_submission_v6_Kfold10_r227_9.csv",index=False)